In [1]:
from src.data.data_retriever import DataRetriever
from src.data.odoo_connector import OdooConnection
import asyncio
from config.settings import settings
import nest_asyncio
import pandas as pd
import numpy as np
from datetime import datetime
nest_asyncio.apply()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
def odoo_missing_values_to_null(df):
    object_cols = df.select_dtypes(include='object').columns
    df[object_cols] = (df[object_cols].replace({False: pd.NA, '' : pd.NA, '/' : pd.NA}))
    df[object_cols] = df[object_cols].map(lambda x: np.nan if x == [] else x)
    return df

In [3]:
def convert_to_datetime(df, columns):
    for col in columns:
        try:
            # formato que no soporta nativamente to_datetime
            if df[col].astype(str).str.contains('/').any():
                df[col] = pd.to_datetime(df[col], errors='coerce', format='%d/%m/%Y')
            else:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        except Exception as e:
            print(f"Error al convertir '{col}': {e}")
    return df

In [4]:
def check_invalid_date_format(series, date_format='%d/%m/%Y'):
    invalid = []
    for val in series.dropna():
        val = str(val).strip()
        if val == '' or val.lower() == 'false':
            continue
        try:
            datetime.strptime(val, date_format)
        except ValueError:
            invalid.append(val)
    return pd.Series(invalid).drop_duplicates().reset_index(drop=True)

# Exploración de los datos

## 1. Estructura de la base de datos

### 1.1. Modelos relevantes

En está sección describo los distintos modelos que deberán ser utilizados. 

También he seleccionado los campos que pueden llegar a ser importantes para el desarrollo. Cabe remarcar que con "importantes", no me refiero únicamente para entrenar el modelo de predicción de impagos, también tengo en cuenta información que el agente podría necesitar.

#### 1.1.1. res.company

Contiene información sobre las empresas que forman el grupo (no clientes):
- id
- name
- currency_id (Identificador de la moneda [id, nombre])

#### 1.1.2. res.partner

Contiene información sobre los partners (clientes/proveedores):
- id
- name
- email
- phone
- street
- city
- zip
- country_id
- customer_rank (>0 es cliente)
- supplier_rank (>0 es proveedor)
- category_id (sector/industria a la que pertenece)
- is_company
- company_type
- company_id
- credit
- credit_limit
- debit
- debit_limit
- industry_id
- invoice_ids
- total_due
- total_invoiced
- total_overdue
- trust
- unpaid_invoice_ids
- unpaid_invoices_count

#### 1.1.3. account.move

Guarda todas las facturas y movimientos contables de la empresa, es decir, los registros de todo lo que se compra, se vende o se paga. Este será el modelo principal con el que trabajará el agente:
- id
- name
- move_type ("out_invoice", "in_invoice", "out_refund", "in_refund", "entry")
- payment_state ("not_paid", "in_payment", "paid", "partial", "reversed")
- company_id
- partner_id
- currency_id
- amount_total
- amount_paid
- amount_residual
- invoice_date
- invoice_date_due
- payment_dates
- date
- create_date
- payment_id
- payment_ids

#### 1.1.4. res.currency

Contiene información sobre las monedas en las que se emiten facturas y se registran los movimientos:
- id
- name
- symbol
- rate

#### 1.1.5. res.country

Contiene información sobre los países:
- id
- name
- code

#### 1.1.6. res.partner.category

Representa las categorías asignadas a los partners:
- id
- name

#### 1.1.7. res.partner.industry

Contiene información de la industria / sector económico de los partners:
- id
- name

### 1.2. Exploración de los datos

#### Conexión a Odoo

In [5]:
odoo_connection = OdooConnection()
asyncio.run(odoo_connection.connect())
data_retriever = DataRetriever(odoo_connection=odoo_connection)

Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


In [6]:
company_df_original = pd.DataFrame(asyncio.run(data_retriever.get_all_companies()))
invoices_df_original = pd.DataFrame(asyncio.run(data_retriever.get_all_outbound_invoices()))
partners_df_original = pd.DataFrame(asyncio.run(data_retriever.get_all_customer_partners()))
currencies_df_original = pd.DataFrame(asyncio.run(data_retriever.get_all_currencies()))
partner_categories_df_original = pd.DataFrame(asyncio.run(data_retriever.get_all_partner_categories()))


Recuperadas 500 facturas, total: 500
Recuperadas 500 facturas, total: 3000
Recuperadas 500 facturas, total: 5500
Recuperadas 500 facturas, total: 8000
Recuperadas 500 facturas, total: 10500
Recuperadas 500 facturas, total: 13000
Recuperadas 500 facturas, total: 15500
Recuperadas 500 facturas, total: 18000
Recuperadas 500 facturas, total: 20500
Recuperadas 500 facturas, total: 23000
Recuperadas 500 facturas, total: 500


Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Ismae\miniconda3\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: cannot enter context: <_contextvars.Context object at 0x000001CC7A3DE9C0> is already entered


In [7]:
company_df_original.to_csv("companies.csv", index=False)
invoices_df_original.to_csv("invoices.csv", index=False)
partners_df_original.to_csv("partners.csv", index=False)
currencies_df_original.to_csv("currencies.csv", index=False)
partner_categories_df_original.to_csv("partner_categories.csv", index=False)

In [123]:
invoices_df = invoices_df_original.copy()
company_df = company_df_original.copy()
partners_df = partners_df_original.copy()
currencies_df = currencies_df_original.copy()
partner_categories_df = partner_categories_df_original.copy()

NameError: name 'company_df_original' is not defined

#### 1.2.1. res.company

In [3]:
company_df

,id,name,currency_id
0,1,"Grupo Viko Digital Marketing, S.A.","[1, EUR]"
1,3,Elogia Media S.L.,"[1, EUR]"
2,2,Ibrands Medios Interactivos SL,"[1, EUR]"
3,5,Kraz Data Solutions SL,"[1, EUR]"
4,6,Marketing4ecommerce Digital Content SL,"[1, EUR]"
5,13,Octoplus Digital Shelf Optimization SL,"[1, EUR]"
6,7,Tandem Trade Marketing SL,"[1, EUR]"
7,11,"DigitalPla2021, S.L.","[1, EUR]"
8,14,Ideas y Estrategia Digital SL,"[1, EUR]"
9,8,INICIATIVAS VIRTUALES DE MEXICO,"[33, MXN]"


El grupo está formado por 12 empresas, 9 usan el euro y 3 el peso mexicano.

#### 1.2.2. account.move

##### Limpieza y procesado básico

In [90]:
invoices_df

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_paid,amount_residual,invoice_date,invoice_date_due,payment_dates,date,create_date,payment_id,payment_ids
0,198507,403-0199881-4444363,out_invoice,not_paid,"[7, Tandem Trade Marketing SL]","[1, Grupo Viko Digital Marketing, S.A.]","[1, EUR]",43.56,0.0,43.56,2025-11-10,2025-11-06,,2025-11-10,2025-11-06 08:35:51,False,[]
1,198549,FVM/2025/00068,out_invoice,not_paid,"[6, Marketing4ecommerce Digital Content SL]","[14913, Ser Sport, S.L.]","[1, EUR]",121.00,0.0,121.00,2025-11-07,2025-12-07,,2025-11-07,2025-11-07 08:23:24,False,[]
2,198548,/,out_invoice,not_paid,"[6, Marketing4ecommerce Digital Content SL]","[14913, Ser Sport, S.L.]","[1, EUR]",726.00,0.0,726.00,False,2025-12-07,,2025-11-07,2025-11-07 08:19:42,False,[]
3,198522,ES501155NOOJRS,out_invoice,not_paid,"[7, Tandem Trade Marketing SL]","[1, Grupo Viko Digital Marketing, S.A.]","[1, EUR]",116.16,0.0,116.16,2025-11-06,2025-11-06,,2025-11-06,2025-11-06 09:20:12,False,[]
4,198516,ES501152NOOJRS,out_invoice,not_paid,"[7, Tandem Trade Marketing SL]","[1, Grupo Viko Digital Marketing, S.A.]","[1, EUR]",43.56,0.0,43.56,2025-11-06,2025-11-06,,2025-11-06,2025-11-06 09:04:18,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24283,1043,FVK/2022/00005,out_invoice,paid,"[5, Kraz Data Solutions SL]","[7812, Salvetti & Llombart, S.L.]","[1, EUR]",242.00,0.0,0.00,2022-10-31,2023-02-20,23/02/2023,2022-10-31,2023-01-11 08:02:48,False,[]
24284,1042,FVK/2022/00004,out_invoice,paid,"[5, Kraz Data Solutions SL]","[7485, Unilever España, S.A.]","[1, EUR]",9075.00,0.0,0.00,2022-10-31,2023-01-02,06/01/2023,2022-10-31,2023-01-11 08:02:48,False,[]
24285,1041,FVK/2022/00003,out_invoice,paid,"[5, Kraz Data Solutions SL]","[7773, Kave Home S.L.]","[1, EUR]",5808.00,0.0,0.00,2022-10-31,2022-12-30,02/01/2023,2022-10-31,2023-01-11 08:02:48,False,[]
24286,1040,FVK/2022/00002,out_invoice,paid,"[5, Kraz Data Solutions SL]","[7409, Ferrer Internacional, S.A.]","[1, EUR]",4961.00,0.0,0.00,2022-10-31,2023-01-25,30/01/2023,2022-10-31,2023-01-11 08:02:48,False,[]


A simple vista se puede apreciar:
- Se deben convertir los valores '', [] y False (en columnas no booleanas) a NA
- Facturas sin nombre o con formatos muy diferentes
- Las últimas facturas aparecen impagadas por ser demasiado recientes
- Parece que hay algunos campos que no tienen fecha de la factura
- payment_id y payment_ids parecen no tener nada

Convierto los valores False / listas vacías a valores NA

In [124]:
invoices_df = odoo_missing_values_to_null(invoices_df)

In [125]:
invoices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24288 entries, 0 to 24287
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                24288 non-null  int64  
 1   name              24248 non-null  object 
 2   move_type         24288 non-null  object 
 3   payment_state     24288 non-null  object 
 4   company_id        24288 non-null  object 
 5   partner_id        24280 non-null  object 
 6   currency_id       24288 non-null  object 
 7   amount_total      24288 non-null  float64
 8   amount_paid       24288 non-null  float64
 9   amount_residual   24288 non-null  float64
 10  invoice_date      24276 non-null  object 
 11  invoice_date_due  24288 non-null  object 
 12  payment_dates     19048 non-null  object 
 13  date              24288 non-null  object 
 14  create_date       24288 non-null  object 
 15  payment_id        24288 non-null  bool   
 16  payment_ids       0 non-null      float6

Analizando los valores null:
- payment_ids es todo null
- Hay 12 facturas sin fecha, 8 sin información sobre el cliente y 40 sin nombre
- Varias facturas sin fecha del pago (impagadas o recientes)

A simple vista, date, create_date, payment_id y payment_ids no son de utilidad, las dos primeras no aportan ninguna información, ya tengo invoice_date, invoice_date_due y payment_dates y las dos últimas son todo False y NA.

In [126]:
invoices_df = invoices_df.drop(columns=['date', 'create_date', 'payment_id', 'payment_ids'])

Las listas company_id, partner_id y currency_id, las separaré en dos columnas cada una

In [127]:
invoices_df['company_name'] = invoices_df['company_id'].apply(lambda x: x[1])
invoices_df['company_id'] = invoices_df['company_id'].apply(lambda x: x[0])

In [128]:
invoices_df['partner_name'] = invoices_df['partner_id'].apply(lambda x: x[1] if isinstance(x, list) else pd.NA)
invoices_df['partner_id'] = invoices_df['partner_id'].apply(lambda x: x[0] if isinstance(x, list) else pd.NA)
invoices_df['currency_name'] = invoices_df['currency_id'].apply(lambda x: x[1])
invoices_df['currency_id'] = invoices_df['currency_id'].apply(lambda x: x[0])

In [129]:
invoices_df.head()

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_paid,amount_residual,invoice_date,invoice_date_due,payment_dates,company_name,partner_name,currency_name
0,198507,403-0199881-4444363,out_invoice,not_paid,7,1,1,43.56,0.0,43.56,2025-11-10,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR
1,198549,FVM/2025/00068,out_invoice,not_paid,6,14913,1,121.00,0.0,121.00,2025-11-07,2025-12-07,<NA>,Marketing4ecommerce Digital Content SL,"Ser Sport, S.L.",EUR
2,198548,<NA>,out_invoice,not_paid,6,14913,1,726.00,0.0,726.00,<NA>,2025-12-07,<NA>,Marketing4ecommerce Digital Content SL,"Ser Sport, S.L.",EUR
3,198522,ES501155NOOJRS,out_invoice,not_paid,7,1,1,116.16,0.0,116.16,2025-11-06,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR
4,198516,ES501152NOOJRS,out_invoice,not_paid,7,1,1,43.56,0.0,43.56,2025-11-06,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR


In [130]:
invoices_df.duplicated().sum()

np.int64(0)

No hay duplicados

In [132]:
invoices_df.nunique()

id                  24288
name                23746
move_type               1
payment_state           5
company_id             12
partner_id           1549
currency_id             6
amount_total         6836
amount_paid             1
amount_residual       861
invoice_date          981
invoice_date_due     1091
payment_dates         817
company_name           12
partner_name         1548
currency_name           6
dtype: int64

Se puede apreciar:
- Varios nombres de facturas vacíos como se ha visto antes
- Aunque las empresas trabajan con dos monedas internamente, han operado con clientes en 6 monedas diferentes, habrá que hacer las conversiones
- Parece que amount_paid no tiene ningún valor (0) por tanto, tampoco aporta ninguna información, amount_residual ya tiene lo que falta por pagar, se puede inferir la cantidad pagada
- Todos los otros campos corresponden a lo esperado

In [134]:
invoices_df['amount_paid'].mean()

np.float64(0.0)

In [135]:
invoices_df = invoices_df.drop(columns=['amount_paid'])

In [143]:
invoices_df[['amount_total', 'amount_residual']].describe()

,amount_total,amount_residual
count,24288.000,24288.000
mean,18940.951,8768.149
std,1159128.047,1140423.340
min,0.000,0.000
25%,12.000,0.000
50%,41.790,0.000
75%,2389.750,0.000
max,177676632.000,177676632.000


Pesos mexicanos, no tiene sentido igualmente factura de casi 8,5 millones de euros

In [ ]:
invoices_df[invoices_df['amount_total'] < invoices_df['amount_total'].quantile(0.001)]

In [139]:
invoices_df['currency_name'].value_counts()

currency_name
EUR    22405
MXN     1628
USD      243
SEK        9
COP        2
GBP        1
Name: count, dtype: int64

In [136]:
invoices_df.head()

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_residual,invoice_date,invoice_date_due,payment_dates,company_name,partner_name,currency_name
0,198507,403-0199881-4444363,out_invoice,not_paid,7,1,1,43.56,43.56,2025-11-10,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR
1,198549,FVM/2025/00068,out_invoice,not_paid,6,14913,1,121.00,121.00,2025-11-07,2025-12-07,<NA>,Marketing4ecommerce Digital Content SL,"Ser Sport, S.L.",EUR
2,198548,<NA>,out_invoice,not_paid,6,14913,1,726.00,726.00,<NA>,2025-12-07,<NA>,Marketing4ecommerce Digital Content SL,"Ser Sport, S.L.",EUR
3,198522,ES501155NOOJRS,out_invoice,not_paid,7,1,1,116.16,116.16,2025-11-06,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR
4,198516,ES501152NOOJRS,out_invoice,not_paid,7,1,1,43.56,43.56,2025-11-06,2025-11-06,<NA>,Tandem Trade Marketing SL,"Grupo Viko Digital Marketing, S.A.",EUR


No parece que hayan muchos campos con valores null:
- **payment_dates**: principalmente de facturas impagadas
- **invoice_date**: algunas facturas sin fecha
- **payment_ids**: no hay ningún dato útil

In [22]:
differences = invoices_df[invoices_df['invoice_date'] != invoices_df['date']]
differences.tail(3)

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_paid,amount_residual,invoice_date,invoice_date_due,payment_dates,date,create_date,payment_id,payment_ids,days_late
24263,13080,INV1/2022/00002,out_invoice,paid,"[8, INICIATIVAS VIRTUALES DE MEXICO]","[8961, ODEM INTERNACIONAL]","[33, MXN]",95120.00,0.0,0.0,2022-12-21,2023-02-19,2023-02-14,2022-12-31,2023-01-27 13:52:02,False,[],-5.0
24269,13244,INV1/2022/00001,out_invoice,reversed,"[9, MITTUM MARKETING RELACIONAL]","[8994, Pagos y Servicios S.A.]","[2, USD]",5411.25,0.0,0.0,2022-12-30,2023-01-29,NaT,2022-12-31,2023-01-27 14:38:02,False,[],NaN
24270,13079,INV1/2022/00001,out_invoice,reversed,"[8, INICIATIVAS VIRTUALES DE MEXICO]","[9392, AXEL DEMB]","[33, MXN]",800.00,0.0,0.0,2022-12-29,2023-02-15,NaT,2022-12-31,2023-01-27 13:52:02,False,[],NaN


No son iguales

In [26]:
n_nulls = invoices_df['payment_dates'].isnull().sum()
n_not_paid = (invoices_df['is_paid'] == False).sum()
print(f"Nulls: {n_nulls} / Not paid: {n_not_paid}")


Nulls: 5350 / Not paid: 5357


In [32]:
not_null_and_not_paid = invoices_df[invoices_df['payment_dates'].notnull() & (invoices_df['is_paid'] == False)]
not_null_and_not_paid['payment_state'].value_counts()

payment_state
in_payment    139
partial        12
Name: count, dtype: int64

In [ ]:
null_and_paid = invoices_df[invoices_df['payment_dates'].notnull() & (invoices_df['is_paid'] == False)]
not_null_and_not_paid

Convierto fechas y creo columna de días de pago tarde:

In [23]:
invoices_df['invoice_date_due'] = pd.to_datetime(invoices_df['invoice_date_due'], errors='coerce', format='%Y-%m-%d')
invoices_df['invoice_date'] = pd.to_datetime(invoices_df['invoice_date'], errors='coerce')
invoices_df['payment_dates'] = pd.to_datetime(invoices_df['payment_dates'], errors='coerce', format='%d/%m/%Y')
invoices_df['date'] = pd.to_datetime(invoices_df['date'], errors='coerce', format='%Y-%m-%d')
invoices_df['create_date'] = pd.to_datetime(invoices_df['create_date'], errors='coerce')
invoices_df['days_late'] = (invoices_df['payment_dates'] - invoices_df['invoice_date_due']).dt.days
invoices_df['is_paid'] = invoices_df['payment_state'] == "paid"

Selecciono los campos relevantes para el análisis:

In [6]:
numerical_cols = invoices_df.select_dtypes(include=['number']).columns.tolist()
categorical_cols = invoices_df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
date_cols = invoices_df.select_dtypes(include=['datetime64']).columns.tolist()

In [7]:
print(f"Tipo fecha: {date_cols}")
print(f"Categóricas: {categorical_cols}")
print(f"Numéricas: {numerical_cols}")

Tipo fecha: ['invoice_date', 'invoice_date_due', 'payment_dates', 'date', 'create_date']
Categóricas: ['name', 'move_type', 'payment_state', 'company_id', 'partner_id', 'currency_id', 'payment_id', 'payment_ids']
Numéricas: ['id', 'amount_total', 'amount_paid', 'amount_residual', 'days_late']


Separo por empresa:

In [59]:
invoices_by_company = {}
for c in company_ids:
    invoices_by_company[c] = invoices_df[invoices_df['company_id'].str[0] == c]

Grupo Viko Digital Marketing, S.A. (1)

In [60]:
invoices_by_company[1]

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_paid,amount_residual,invoice_date,invoice_date_due,payment_dates,date,create_date,payment_id,payment_ids,company_id_id,partner_id_id,days_late
1165,197174,FACT-2025-XYZ,out_invoice,not_paid,"[1, Grupo Viko Digital Marketing, S.A.]",False,"[1, EUR]",0.00,0.0,0.0,False,2025-07-23,NaT,2025-07-23,2025-07-23 08:54:56,False,[],1,NaN,NaN
1166,197173,FACT-2025-XYZ,out_invoice,not_paid,"[1, Grupo Viko Digital Marketing, S.A.]",False,"[1, EUR]",0.00,0.0,0.0,False,2025-07-23,NaT,2025-07-23,2025-07-23 08:54:47,False,[],1,NaN,NaN
1167,197172,FACT-2025-XYZ,out_invoice,not_paid,"[1, Grupo Viko Digital Marketing, S.A.]","[123, María Sieiro Alfonsin]","[1, EUR]",484.00,0.0,484.0,2025-07-23,2025-07-23,NaT,2025-07-23,2025-07-23 08:50:42,False,[],1,123.0,NaN
1168,197171,FACT-2025-XYZ,out_invoice,not_paid,"[1, Grupo Viko Digital Marketing, S.A.]","[123, María Sieiro Alfonsin]","[1, EUR]",484.00,0.0,484.0,2025-07-23,2025-07-23,NaT,2025-07-23,2025-07-23 08:50:04,False,[],1,123.0,NaN
1765,196522,FVV/2025/00001,out_invoice,not_paid,"[1, Grupo Viko Digital Marketing, S.A.]","[8571, Google Ireland Limited]","[1, EUR]",10.00,0.0,10.0,False,2025-06-05,NaT,2025-06-05,2025-06-05 13:56:40,False,[],1,8571.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24246,12479,INV1/2022/00005,out_invoice,reversed,"[1, Grupo Viko Digital Marketing, S.A.]","[15, INICIATIVAS VIRTUALES DE MEXICO]","[1, EUR]",2122.40,0.0,0.0,2022-11-30,2023-01-29,NaT,2022-12-31,2023-01-26 15:29:31,False,[],1,15.0,NaN
24252,12478,INV1/2022/00004,out_invoice,paid,"[1, Grupo Viko Digital Marketing, S.A.]","[7749, Property Technology Services, S.L.]","[1, EUR]",544.50,0.0,0.0,2022-12-31,2023-01-22,2023-01-25,2022-12-31,2023-01-26 15:29:31,False,[],1,7749.0,3.0
24258,12477,INV1/2022/00003,out_invoice,paid,"[1, Grupo Viko Digital Marketing, S.A.]","[9250, Equipzilla, S.L.]","[1, EUR]",181.50,0.0,0.0,2022-12-31,2023-01-22,2023-01-13,2022-12-31,2023-01-26 15:29:31,False,[],1,9250.0,-9.0
24264,12476,INV1/2022/00002,out_invoice,paid,"[1, Grupo Viko Digital Marketing, S.A.]","[8048, Lanai Capital Partners, S.L.]","[1, EUR]",145.03,0.0,0.0,2022-12-31,2023-01-30,2023-06-06,2022-12-31,2023-01-26 15:29:31,False,[],1,8048.0,127.0


Elogia Media S.L. (3)

Ibrands Medios Interactivos SL (2)

Kraz Data Solutions SL (5)

Marketing4ecommerce Digital Content SL (6)

Octoplus Digital Shelf Optimization SL (13)

Tandem Trade Marketing SL (7)

DigitalPla2021, S.L. (11)

Ideas y Estrategia Digital SL (14)

INICIATIVAS VIRTUALES DE MEXICO	(8)

IBRANDS MEDIOS INTERACTIVOS DE MEXICO (12)

Creo dos columnas nuevas con únicamente el id de la empresa y del partner en la factura

Filtro las facturas por empresa:

In [24]:
partners_df['id']

TypeError: list indices must be integers or slices, not str

In [12]:
invoices_df['company_id'].value_counts()

company_id
[7, Tandem Trade Marketing SL]                  12429
[3, Elogia Media S.L.]                           6348
[8, INICIATIVAS VIRTUALES DE MEXICO]             1617
[6, Marketing4ecommerce Digital Content SL]      1184
[11, DigitalPla2021, S.L.]                        785
[2, Ibrands Medios Interactivos SL]               724
[14, Ideas y Estrategia Digital SL]               595
[13, Octoplus Digital Shelf Optimization SL]      194
[1, Grupo Viko Digital Marketing, S.A.]           140
[9, MITTUM MARKETING RELACIONAL]                  115
[5, Kraz Data Solutions SL]                       114
[12, IBRANDS MEDIOS INTERACTIVOS DE MEXICO]        43
Name: count, dtype: int64

#### 1.2.2. res.partner

In [33]:
partners_df = pd.DataFrame(asyncio.run(data_retriever.get_all_customer_partners()))


Recuperadas 500 facturas, total: 500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1631 non-null   int64  
 1   name                   1631 non-null   object 
 2   email                  1631 non-null   object 
 3   phone                  1631 non-null   object 
 4   street                 1631 non-null   object 
 5   city                   1631 non-null   object 
 6   zip                    1631 non-null   object 
 7   country_id             1631 non-null   object 
 8   customer_rank          1631 non-null   int64  
 9   supplier_rank          1631 non-null   int64  
 10  category_id            1631 non-null   object 
 11  is_company             1631 non-null   bool   
 12  company_type           1631 non-null   object 
 13  company_id             1631 non-null   object 
 14  credit             

In [35]:
partners_df_original = partners_df.copy()

In [51]:
partners_df = partners_df_original.copy()

In [66]:
invoices_df_original.dtypes

id                    int64
name                 object
move_type            object
payment_state        object
company_id           object
partner_id           object
currency_id          object
amount_total        float64
amount_paid         float64
amount_residual     float64
invoice_date         object
invoice_date_due     object
payment_dates        object
date                 object
create_date          object
payment_id             bool
payment_ids          object
dtype: object

In [54]:
object_cols = partners_df.select_dtypes(include='object').columns
partners_df[object_cols] = (partners_df[object_cols].replace({False: pd.NA}))

In [52]:
partners_df[object_cols] = partners_df[object_cols].map(lambda x: np.nan if x == [] else x)

In [55]:
partners_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1631 non-null   int64  
 1   name                   1630 non-null   object 
 2   email                  281 non-null    object 
 3   phone                  12 non-null     object 
 4   street                 1568 non-null   object 
 5   city                   1549 non-null   object 
 6   zip                    1551 non-null   object 
 7   country_id             1616 non-null   object 
 8   customer_rank          1631 non-null   int64  
 9   supplier_rank          1631 non-null   int64  
 10  category_id            284 non-null    object 
 11  is_company             1631 non-null   bool   
 12  company_type           1631 non-null   object 
 13  company_id             569 non-null    object 
 14  credit                 1631 non-null   float64
 15  cred

In [61]:
partners_df[partners_df['is_company'] == False]

,id,name,email,phone,street,city,zip,country_id,customer_rank,supplier_rank,...,debit,debit_limit,industry_id,invoice_ids,total_due,total_invoiced,total_overdue,trust,unpaid_invoice_ids,unpaid_invoices_count
8,10970,Aaron Escobar,esaaroleesco@icloud.com,<NA>,Cuauhtemoc 123,Ciudad de México,16090,"[156, Mexico]",2,0,...,0.0,0.0,<NA>,"[63361, 63691, 55911]",0.0,39.98,0.0,normal,NaN,0
12,12023,Abel Hernández,abel@congresomarketingdigital.com,<NA>,Caldas Da Raihna 6,Badajoz,6011,"[68, Spain]",1,0,...,0.0,0.0,<NA>,"[77026, 93888]",0.0,199.00,0.0,normal,NaN,0
15,9405,Facturacion,facturacion@aby.group,<NA>,"Hijas de la Caridad, 108",Bilbao,48009,"[68, Spain]",2,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
20,10178,Luís Granados,lgranados@acesur.com,<NA>,Carretera de la carolina,Vilches,23220,"[68, Spain]",1,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
34,13148,Invoice,invoice@adock.io,<NA>,"Calle Cardenal Vives i Tutó, 65",Barcelona,08034,"[68, Spain]",1,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,13665,Teresa,teresa@growwer.com,<NA>,"Carretera d'Esplugues 47, Esc. D, 5 - 1",Cornellà del Llobregat,08940,"[68, Spain]",1,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
1569,13577,"Vitola Marketing, S.L.",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
1580,13664,Celeste Romero,celeste.arias@vtex.com,<NA>,"WeWork Aviation House, 125 Kingsway",London,WC2B 6NH,"[231, United Kingdom]",1,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0
1604,13788,Sebastián Díaz,sebastiandiaz@wix.com,<NA>,"40 Hanamal Tel Aviv, Beit Yoel",Tel Aviv,6350671,"[102, Israel]",1,0,...,0.0,0.0,<NA>,NaN,0.0,0.00,0.0,normal,NaN,0


In [56]:
partners_df

,id,name,email,phone,street,city,zip,country_id,customer_rank,supplier_rank,...,debit,debit_limit,industry_id,invoice_ids,total_due,total_invoiced,total_overdue,trust,unpaid_invoice_ids,unpaid_invoices_count
0,14516,200 Labs Inc,<NA>,<NA>,"490 Post St, Ste 526",San Francisco,94102,"[233, United States]",1,5,...,600.0,0.0,<NA>,"[196646, 196260, 197176, 196645, 196263, 19664...",0.0,1808.50,0.0,normal,NaN,0
1,12500,"202 Digital Reputation, S.L.",<NA>,<NA>,"C/ Tuset 19, entresuelo",Barcelona,08006,"[68, Spain]",3,0,...,0.0,0.0,<NA>,"[101525, 100842, 100844, 100079]",0.0,600.00,0.0,normal,NaN,0
2,10577,"2Be Confirmed Events, S.L.",<NA>,<NA>,"C/ Doctor Fleming, 36",Madrid,28036,"[68, Spain]",1,0,...,0.0,0.0,<NA>,[46745],0.0,500.00,0.0,normal,NaN,0
3,12233,"2BeGroup&Partners, S.L.",<NA>,<NA>,"Rua Das Baleras, 13 - 4 Oficina 5",Santiago de Compostela,15705,"[68, Spain]",3,0,...,0.0,0.0,<NA>,"[162545, 150132, 83140, 83139]",0.0,1871.90,0.0,normal,NaN,0
4,14539,"2 Open EU CN, S.L.U.",<NA>,<NA>,"C/Sierpes, 3",Cáceres,10003,"[68, Spain]",1,0,...,0.0,0.0,<NA>,"[166755, 166006]",0.0,600.00,0.0,normal,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,14891,ZENVIA MEXICO,<NA>,<NA>,LAS PRADERAS 12 - PISO 1 CUBICULO A,Coyoacán,04500,"[156, Mexico]",1,0,...,0.0,0.0,<NA>,"[185677, 185678, 185676]",0.0,5264.02,0.0,normal,NaN,0
1627,9285,Zippy - Comercio e Distribuição SA,<NA>,<NA>,"Rua João Mendonça, nº 529",Senhora da Hora,4464-503,"[183, Portugal]",1,0,...,0.0,0.0,<NA>,"[13463, 11873]",0.0,2650.00,0.0,normal,NaN,0
1628,11067,"Zoconet, S.L.",<NA>,<NA>,"Avd. Juan López Peñalver, 17",Málaga,29590,"[68, Spain]",1,0,...,0.0,0.0,<NA>,[58839],0.0,25.00,0.0,normal,NaN,0
1629,14429,"Zumitow, S.L.",<NA>,<NA>,"C/Pensamiento 27, Pta. 3, Esc. Izq.. Plt. 3",Madrid,28020,"[68, Spain]",1,0,...,0.0,0.0,<NA>,"[160887, 155371]",0.0,1000.00,0.0,normal,NaN,0


A simple vista, se puede observar que hay muchos valores vacíos (False en Odoo) y campos vacíos ([] en category_id).

Analizaré los clientes de cada empresa:

In [6]:
partners_df['company_id'].value_counts()

company_id
False                                           1062
[7, Tandem Trade Marketing SL]                   179
[11, DigitalPla2021, S.L.]                       154
[8, INICIATIVAS VIRTUALES DE MEXICO]             153
[6, Marketing4ecommerce Digital Content SL]       29
[14, Ideas y Estrategia Digital SL]               18
[3, Elogia Media S.L.]                            17
[12, IBRANDS MEDIOS INTERACTIVOS DE MEXICO]        6
[13, Octoplus Digital Shelf Optimization SL]       6
[9, MITTUM MARKETING RELACIONAL]                   4
[2, Ibrands Medios Interactivos SL]                3
Name: count, dtype: int64

La mayoría de partners no tienen asociadas las empresas con las que han hecho transacciones...

Lo sacaré de account.move

In [28]:
company_ids = [1,2,3,5,6,7,8,9,11,12,13,14]
def get_partners_by_company(company_id):
    invoices_id = invoices_df[invoices_df['company_id_id'] == 1]
    partners_id = invoices_id['partner_id_id'].unique()
    return partners_df[partners_df['id'].isin(partners_id)]

In [29]:
partners_by_company = []
for c in company_ids:
    partners_by_company.append({c : get_partners_by_company(c)})

In [ ]:
invoices_df['company_id_id'] = invoices_df['company_id'].str[0]
invoices_df['partner_id_id'] = invoices_df['partner_id'].str[0]

In [38]:
company_ids = [1,2,3,5,6,7,8,9,11,12,13,14]
invoices_by_company = {}
for c in company_ids:
    invoices_by_company[c] = invoices_df[invoices_df['company_id'].str[0] == c]

Grupo Viko Digital Marketing, S.A. (1)

In [39]:
invoices_by_company[1]['partner_id'].value_counts()

partner_id
[9250, Equipzilla, S.L.]                           18
[10, Elogia Media S.L.]                            17
[11527, The Tropicfeel S.L.]                       12
[7749, Property Technology Services, S.L.]         10
[13, Marketing4ecommerce Digital Content SL]        6
[8527, Fundació Pasqual Maragall]                   6
[12, Kraz Data Solutions SL]                        6
[16, MITTUM MARKETING RELACIONAL]                   5
[10436, Octoplus Digital Shelf Optimization SL]     5
[12322, Byfacility, S.L.]                           5
[15, INICIATIVAS VIRTUALES DE MEXICO]               4
[13260, Ideas y Estrategia Digital SL]              3
[8307, Carglass, S.L.U.]                            3
[18, DigitalPla2021, S.L.]                          3
[12548, Barkibu S.L.]                               2
[12867, Ufinet Latam, S.L.U.]                       2
False                                               2
[123, María Sieiro Alfonsin]                        2
[14, Tandem Trade

Elogia Media S.L. (3)

In [54]:
print(invoices_by_company[3]['partner_id'].count())
invoices_by_company[3]['partner_id'].value_counts()

6348


partner_id
[8920, Boehringer Ingelheim España, S.A.]       573
[8679, Fira Internacional de Barcelona (ES)]    351
[8527, Fundació Pasqual Maragall]               231
[9308, AMAZON INVOICING SPAIN]                  201
[8870, Hero España SA]                          190
                                               ... 
[7758, Laboratorios Niam, S.L.]                   1
[8170, Bcnscience, S.L.]                          1
[7645, Galicia Sport 360 SLU]                     1
[7432, Naturgy Iberia, S.A.]                      1
[8003, Gree Products, S.L.]                       1
Name: count, Length: 296, dtype: int64

Ibrands Medios Interactivos SL (2)

In [42]:
invoices_by_company[2]['partner_id'].value_counts()

partner_id
[10, Elogia Media S.L.]                      72
[7907, Aby Marketing Dreams, S.L.]           35
[9311, Cint AB]                              30
[7854, Merkal Calzados S.L.]                 28
[7654, Feebbo Solutions, S.L.]               25
                                             ..
[8415, Kokoen GmbH]                           1
[8895, Datawork Marketing SL]                 1
[7481, Prosegur Compañía de Seguridad SA]     1
[7519, Roman y Asociados S.A.]                1
[9309, Diario ABC, S.L.]                      1
Name: count, Length: 96, dtype: int64

Kraz Data Solutions SL (5)

In [43]:
invoices_by_company[5]['partner_id'].value_counts()

partner_id
[10, Elogia Media S.L.]                                       16
[7812, Salvetti & Llombart, S.L.]                              9
[7773, Kave Home S.L.]                                         8
[7485, Unilever España, S.A.]                                  8
[1, Grupo Viko Digital Marketing, S.A.]                        7
[7419, Editorial Planeta, S.A.U.]                              7
[7461, Joyeria Tous S.A]                                       7
[7479, Vinoselección, S.A.]                                    6
[12541, SEAT, S.A.]                                            6
[10993, Pikostore, S.L.U.]                                     5
[10994, Pikolinos Intercontinental, S.A.]                      5
[8679, Fira Internacional de Barcelona (ES)]                   4
[10436, Octoplus Digital Shelf Optimization SL]                4
[13766, RuralMed, S.L.]                                        3
[7409, Ferrer Internacional, S.A.]                             3
[7421, Saba Ap

Marketing4ecommerce Digital Content SL (6)

In [44]:
invoices_by_company[6]['partner_id'].value_counts()

partner_id
[8571, Google Ireland Limited]            39
[9277, Pipedrive Inc]                     25
[8012, Virality Media, S.L.]              23
[9665, Pixel Labs LLC]                    22
[9605, Getlinko International, S.L.]      21
                                          ..
[14923, Ceramic Connection Shop, S.L.]     1
[14912, Santafixie Group, S.L.]            1
[14911, Channelbook, S.L.U.]               1
[15057, Flyeralarm, S.L.]                  1
[8527, Fundació Pasqual Maragall]          1
Name: count, Length: 472, dtype: int64

Octoplus Digital Shelf Optimization SL (13)

In [45]:
invoices_by_company[13]['partner_id'].value_counts()

partner_id
[10590, Beam Suntory Distribution SL]                    98
[10587, Beam Inc. Global Business Services]              62
[7403, Nestlé España S.A.]                                5
[10598, Medios Activos y Aplicacion de Servicios S.L]     4
[10595, Ceva Sante Animale]                               4
[12060, Beam Suntory España Beverages, S.L.U.]            3
[10616, Beautyge, S.L.]                                   3
[8062, Optopus Optimisation, S.L.]                        2
[14768, Beam Suntory Asia Pte. Ltd.]                      2
[10858, Beam Canada Inc.]                                 2
[14722, Colgate-Palmolive España, S.A.]                   2
[10594, Beam Suntory Australia Pty Ltd]                   2
[13260, Ideas y Estrategia Digital SL]                    1
[10592, Beam Suntory Germany GmbH]                        1
[11484, Casa Santiveri, S.L.]                             1
[10589, Beam Suntory Spain S.L]                           1
[10588, Cuetara S.L.U]       

Tandem Trade Marketing SL (7)

In [46]:
invoices_by_company[7]['partner_id'].value_counts()

partner_id
[10892, Marketplaces España]                 7606
[13926, Marketplaces Italia]                 2742
[13924, Marketplaces Francia]                 499
[14, Tandem Trade Marketing SL]               326
[15076, pruebas-cif-12345]                    190
                                             ... 
[10087, José Carlos Rodríguez Diago]            1
[10331, Inmopanta. S.L.]                        1
[10332, TecniOrganic, S.L.U.]                   1
[10333, La Costanera Santa Eulalia, S.L.]       1
[15041, Nicolás Sánchez-Biezma]                 1
Name: count, Length: 296, dtype: int64

DigitalPla2021, S.L. (11)

In [47]:
invoices_by_company[11]['partner_id'].value_counts()

partner_id
[9902, Manuel Alejandro Mesa Sánchez]    52
[9913, Sara Carbajo]                     26
[9881, Betty Lepina]                     26
[9916, Tomasz Smardzewski]               26
[9901, Paula Garcia Bustos]              26
                                         ..
[9887, Lola Garau]                        1
[9896, ISABEL ORGAZ TARAVILLA]            1
[9886, Daniel Lopez]                      1
[9883, Carmen Luz Zarrías Villena]        1
[9917, Valentin Salas]                    1
Name: count, Length: 233, dtype: int64

Ideas y Estrategia Digital SL (14)

In [48]:
invoices_by_company[14]['partner_id'].value_counts()

partner_id
[13557, Chiesi España, S.A]                          124
[13548, Esteve Pharmaceuticals SA]                   113
[13558, Elanco Spain S.L.]                           106
[8933, Zambon, S.A.U.]                                66
[13559, Kern Pharma S.L.]                             47
[13549, Swedish Orphan Biovitrum S.L.]                34
[13542, Laboratorio Reig Jofre, S.A.]                 21
[13547, Alexion Pharma Nordics AB]                    18
[13543, Alexion Pharma GMBH]                          11
[14595, Swedish Orphan Biovitrum AB (publ)]            8
[7583, Reckitt Benckiser Healthcare, S.A.]             8
[14476, Faes Farma S.A.]                               7
[13654, Atika Pharma S.L.]                             7
[13545, Elanco AH Portugal Unipessoal Lda.]            6
[13544, Angelini Pharma Portugal, Unipessoal Lda]      3
[13553, AdSalutem Lullaai S.L]                         2
[14627, Asociación Española Contra el Cáncer]          2
[10, Elogia Media S.

INICIATIVAS VIRTUALES DE MEXICO	(8)

In [49]:
invoices_by_company[8]['partner_id'].value_counts()

partner_id
[12056, COMERCIALIZADORA ALMACENES GARCIA DE MEXICO]    69
[9678, DAWN-MIXCO INTERNACIONAL]                        66
[8960, SEPHORA MEXICO]                                  60
[11041, TOYOTA TSUSHO CORPORATION DE MEXICO]            47
[9759, VCD CONSTRUCCION Y DESARROLLO]                   44
                                                        ..
[9391, RAGS]                                             1
[9390, SERVICIOS DE CAPITAL HUMANO AXO]                  1
[9389, MR BON MEXICO]                                    1
[9388, PULQUEDIGITAL]                                    1
[9392, AXEL DEMB]                                        1
Name: count, Length: 172, dtype: int64

IBRANDS MEDIOS INTERACTIVOS DE MEXICO (12)

In [50]:
invoices_by_company[12]['partner_id'].value_counts()

partner_id
[8987, ASALES, SERVICIOS DE MARKETING EN INTERNET]    30
[9387, TIENDAS SORIANA]                                5
[11708, ANTEVENIO MEXICO]                              4
[13917, KOVAFINANSI AMERICAS CORPORATE]                1
[16, MITTUM MARKETING RELACIONAL]                      1
[9427, SILVERSPRINGS SERVICIOS CORPORATIVOS]           1
[9386, INDUSTRIAS TUK]                                 1
Name: count, dtype: int64

MITTUM MARKETING RELACIONAL (9)

In [51]:
invoices_by_company[9]['partner_id'].value_counts()

partner_id
[8991, SEGUROS BANAMEX]                           30
[8989, TARJETAS BANAMEX]                          28
[8993, BANCO NACIONAL DE MEXICO]                  26
[9797, SOCIEDAD COOPERATIVA DE CONSUMO PEMEX,]    20
[8994, Pagos y Servicios S.A.]                     6
[8992, EVO PAYMENTS MEXICO.]                       5
Name: count, dtype: int64

##### Conclusiones

Hay diferencias significativas entre cada empresa:
- **Tandem Trade Marketing**: Tiene el mayor nombre de facturas (más de 10.000), la mayoría concentradas en Markeplace. Son de importe bajo y de alta frecuencia y se cobran inmediatamente, por tanto, tienen un riesgo de impago prácticamente nulo y pueden afectar negativamente al modelo de predicción aportando un sesgo positivo.
- **Elogia Media**: facturación B2B con importes medio/altos

# MVP

Comenzaré realizando una versión mínima viable del modelo de predicción de impagos.

Para facilitar el proceso, este modelo entrenará a partir de las facturas y podrá predecir si una factura va a ser pagada a tiempo o tarde.

In [ ]:
!pip install pydantic[email]

In [ ]:
pip install mcp_odoo/.

In [7]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


# Datos empresas

In [15]:
asyncio.run(data_retriever.get_all_companies())

[Company(id=1, name='Grupo Viko Digital Marketing, S.A.', currency_id=(1, 'EUR')),
 Company(id=3, name='Elogia Media S.L.', currency_id=(1, 'EUR')),
 Company(id=2, name='Ibrands Medios Interactivos SL', currency_id=(1, 'EUR')),
 Company(id=5, name='Kraz Data Solutions SL', currency_id=(1, 'EUR')),
 Company(id=6, name='Marketing4ecommerce Digital Content SL', currency_id=(1, 'EUR')),
 Company(id=13, name='Octoplus Digital Shelf Optimization SL', currency_id=(1, 'EUR')),
 Company(id=7, name='Tandem Trade Marketing SL', currency_id=(1, 'EUR')),
 Company(id=11, name='DigitalPla2021, S.L.', currency_id=(1, 'EUR')),
 Company(id=14, name='Ideas y Estrategia Digital SL', currency_id=(1, 'EUR')),
 Company(id=8, name='INICIATIVAS VIRTUALES DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=12, name='IBRANDS MEDIOS INTERACTIVOS DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=9, name='MITTUM MARKETING RELACIONAL', currency_id=(33, 'MXN'))]

In [12]:
fields = [
        "id",
        "name",
        "email",
        "phone",
        "street",
        "city",
        "zip",
        "country_id",
        "customer_rank",
        "supplier_rank",
        "category_id",
        "company_type",
        "credit",
        "credit_limit",
        "debit",
        "debit_limit",
        "industry_id",
        "invoice_ids",
        "total_due",
        "total_invoiced",
        "total_overdue",
        "trust",
        "unpaid_invoice_ids",
        "unpaid_invoices_count",
    ]

partners = await odoo_connection.search_read(
        model="res.partner",
        domain=[("customer_rank", ">", 0)],
        fields=fields,
        limit=1000,
        offset=0
    )

In [24]:
payments = await odoo_connection.search_read(
        model="res.partner",
        domain=[("payment_type", "=", "inbound")],
        fields=[],
        limit=1000,
        offset=0
    )

OdooConnectionError: Error executing search_read on res.partner: <Fault 1: 'Traceback (most recent call last):\n  File "/opt/odoo/odoo/odoo/addons/base/controllers/rpc.py", line 151, in xmlrpc_2\n    response = self._xmlrpc(service)\n               ^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/addons/base/controllers/rpc.py", line 127, in _xmlrpc\n    result = dispatch_rpc(service, method, params)\n             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/http.py", line 369, in dispatch_rpc\n    return dispatch(method, params)\n           ^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 56, in dispatch\n    res = execute_kw(db, uid, *params[3:])\n          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 79, in execute_kw\n    return execute(db, uid, obj, method, *args, **kw or {})\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/odoo/odoo/odoo/service/model.py", line 

In [20]:
count_inbound = await odoo_connection.execute_kw(
    model="account.payment",
    method="search_count",
    args=[[("payment_type", "=", "outbound")]]
)

# Datos facturas

In [32]:
company_id = 14
invoices = asyncio.run(data_retriever.get_all_outbound_invoices(company_id))
invoices_df = pd.DataFrame([i.to_dict() for i in invoices])

Recuperadas 500 facturas, total: 500
Recuperadas 95 facturas, total: 595


In [33]:
invoices_df.describe()

,id,amount_total,amount_residual,days_overdue
count,595.000000,595.000000,595.000000,595.000000
mean,143420.842017,5338.248672,912.600571,7.858824
std,21899.350306,8546.715734,4151.656529,13.038414
min,121169.000000,108.730000,0.000000,-1.000000
25%,130328.500000,955.900000,0.000000,0.000000
50%,130477.000000,2178.000000,0.000000,4.000000
75%,157948.500000,6198.225000,0.000000,11.500000
max,194734.000000,90096.600000,41964.870000,103.000000


In [34]:
invoices_df.columns.values

array(['id', 'name', 'move_type', 'payment_state', 'company_id',
       'partner_id', 'currency_id', 'amount_total', 'amount_residual',
       'invoice_date', 'invoice_date_due', 'journal_id', 'payment_dates',
       'paid_late', 'days_overdue'], dtype=object)

In [35]:
invoices_df["payment_dates"].dtypes

dtype('O')

In [36]:
invoices_df["payment_state"].value_counts()

payment_state
paid        515
not_paid     80
Name: count, dtype: int64

In [37]:
invoices_df["paid_late"].value_counts()

paid_late
True     365
False    150
Name: count, dtype: int64

In [38]:
invoices_df["partner_id"].value_counts()

partner_id
(13557, Chiesi España, S.A)                          124
(13548, Esteve Pharmaceuticals SA)                   113
(13558, Elanco Spain S.L.)                           106
(8933, Zambon, S.A.U.)                                66
(13559, Kern Pharma S.L.)                             47
(13549, Swedish Orphan Biovitrum S.L.)                34
(13542, Laboratorio Reig Jofre, S.A.)                 21
(13547, Alexion Pharma Nordics AB)                    18
(13543, Alexion Pharma GMBH)                          11
(14595, Swedish Orphan Biovitrum AB (publ))            8
(7583, Reckitt Benckiser Healthcare, S.A.)             8
(14476, Faes Farma S.A.)                               7
(13654, Atika Pharma S.L.)                             7
(13545, Elanco AH Portugal Unipessoal Lda.)            6
(13544, Angelini Pharma Portugal, Unipessoal Lda)      3
(13553, AdSalutem Lullaai S.L)                         2
(14627, Asociación Española Contra el Cáncer)          2
(10, Elogia Media S.

In [16]:
invoices_df["currency_id"].value_counts()

currency_id
(1, EUR)      12388
(18, SEK)         9
(142, GBP)        1
Name: count, dtype: int64

In [10]:
invoices_df

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_residual,invoice_date,invoice_date_due,journal_id,payment_dates,paid_late,days_overdue
0,196743,FVE/2025/00376,out_invoice,not_paid,"(3, Elogia Media S.L.)","(8920, Boehringer Ingelheim España, S.A.)","(1, EUR)",242.00,242.00,2025-06-17,2025-08-16,"(28, Facturas ventas Elogia)",None,None,-1
1,196654,FVE/2025/00375,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",453.75,453.75,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
2,196649,FVE/2025/00374,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",907.50,907.50,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
3,196631,FVE/2025/00373,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
4,196628,FVE/2025/00372,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,9691,INV1/2022/00005,out_invoice,paid,"(3, Elogia Media S.L.)","(9247, Apoteca Natura Spa)","(1, EUR)",2250.00,0.00,2022-12-31,2023-01-30,"(340, Carga facturas venta Elogia)",2023-02-01,True,2
6344,9690,INV1/2022/00004,out_invoice,paid,"(3, Elogia Media S.L.)","(8301, Goiko Grill Group SL.)","(1, EUR)",11555.50,0.00,2022-12-31,2023-01-29,"(340, Carga facturas venta Elogia)",2023-02-17,True,19
6345,9689,INV1/2022/00003,out_invoice,paid,"(3, Elogia Media S.L.)","(7439, Boehringer Ingelheim Animal Health Espa...","(1, EUR)",1597.20,0.00,2022-12-31,2023-02-05,"(340, Carga facturas venta Elogia)",2023-03-01,True,24
6346,9688,INV1/2022/00002,out_invoice,paid,"(3, Elogia Media S.L.)","(8663, Irmaos Vila Nova SA)","(1, EUR)",12195.41,0.00,2022-12-31,2023-02-18,"(340, Carga facturas venta Elogia)",2023-02-21,True,3
